In [1]:
!apt update -qq
!apt install -qq postgresql postgresql-contrib

36 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
postgresql is already the newest version (14+238).
postgresql-contrib is already the newest version (14+238).
0 upgraded, 0 newly installed, 0 to remove and 36 not upgraded.


In [2]:
# Start PostgreSQL explicitly
!service postgresql start

 * Starting PostgreSQL 14 database server
   ...done.


In [3]:
%cd /content

/content


In [4]:
# Setup PostgreSQL User and Database
!sudo -u postgres psql -c "CREATE USER colab WITH PASSWORD 'colab';"
!sudo -u postgres psql -c "CREATE DATABASE bank_reviews OWNER colab;"
!sudo -u postgres psql -c "GRANT ALL PRIVILEGES ON DATABASE bank_reviews TO colab;"

ERROR:  role "colab" already exists
ERROR:  database "bank_reviews" already exists
GRANT


In [5]:
# Install necessary Python package
!pip install psycopg2-binary

import psycopg2
import pandas as pd

# Explicit PostgreSQL connection
conn = psycopg2.connect(
    dbname="bank_reviews",
    user="colab",
    password="colab",
    host="localhost",
    port="5432"
)
cursor = conn.cursor()


In [11]:
# Create Banks table explicitly
# Rollback any failed transactions before creating tables
conn.rollback()
cursor.execute("""
CREATE TABLE IF NOT EXISTS banks (
    bank_id SERIAL PRIMARY KEY,
    bank_name VARCHAR(100) UNIQUE NOT NULL
);
""")

# Create Reviews table explicitly - Drop if exists to apply schema changes
cursor.execute("DROP TABLE IF EXISTS reviews;")
cursor.execute("""
CREATE TABLE IF NOT EXISTS reviews (
    review_id SERIAL PRIMARY KEY,
    bank_id INTEGER REFERENCES banks(bank_id),
    review_text TEXT NOT NULL,
    rating INTEGER NOT NULL,
    review_date DATE NOT NULL,
    sentiment_label VARCHAR(20),
    sentiment_score FLOAT,
    theme VARCHAR(255), -- Increased length to accommodate longer strings
    source VARCHAR(50)
);
""")

conn.commit()

In [12]:
%cd

/root


In [13]:
# Load your data
df_reviews = pd.read_csv('/content/drive/MyDrive/Tenx program/week-2/data/reviews_analyzed.csv')

# Insert unique banks explicitly
banks = df_reviews['bank'].unique()

for bank in banks:
    cursor.execute("INSERT INTO banks (bank_name) VALUES (%s) ON CONFLICT (bank_name) DO NOTHING;", (bank,))
conn.commit()

# Fetch bank IDs explicitly
cursor.execute("SELECT bank_id, bank_name FROM banks;")
bank_id_mapping = {name: bank_id for bank_id, name in cursor.fetchall()}

# Insert reviews explicitly
for _, row in df_reviews.iterrows():
    cursor.execute("""
    INSERT INTO reviews (bank_id, review_text, rating, review_date, sentiment_label, sentiment_score, theme, source)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
    """, (
        bank_id_mapping[row['bank']],
        row['review'],
        row['rating'],
        row['date'],
        row.get('sentiment_label'),
        row.get('sentiment_score'),
        row.get('theme'),
        row['source']
    ))

conn.commit()
cursor.close()
conn.close()

In [18]:
# Export database explicitly to the 'database' folder
!PGPASSWORD='colab' pg_dump -U colab -h localhost bank_reviews > "/content/drive/MyDrive/Tenx program/week-2/database/bank_reviews_dump.sql"


In [20]:
# Check explicitly that dump file is saved
!ls -l '/content/drive/MyDrive/Tenx program/week-2/database'

total 148
-rw------- 1 root root 151514 Jun  8 08:15 bank_reviews_dump.sql


In [21]:
import psycopg2
import pandas as pd

# Connect explicitly
conn = psycopg2.connect(
    dbname="bank_reviews",
    user="colab",
    password="colab",
    host="localhost",
    port="5432"
)


In [22]:
# Query data explicitly from reviews table
query = """
SELECT r.review_id, b.bank_name, r.review_text, r.rating, r.review_date,
       r.sentiment_label, r.sentiment_score, r.theme, r.source
FROM reviews r
JOIN banks b ON r.bank_id = b.bank_id;
"""

# Load into DataFrame explicitly
df_from_db = pd.read_sql_query(query, conn)

# Check explicitly
df_from_db.head()


<ipython-input-22-546570124e32>:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_from_db = pd.read_sql_query(query, conn)


,review_id,bank_name,review_text,rating,review_date,sentiment_label,sentiment_score,theme,source
0,1,CBE,"""Why don’t your ATMs support account-to-accoun...",4,2025-06-06,NEGATIVE,0.996465,"Customer Support, Financial Transactions",Google Play
1,2,CBE,what is this app problem???,1,2025-06-05,NEGATIVE,0.999623,Other,Google Play
2,3,CBE,the app is proactive and a good connections.,5,2025-06-05,POSITIVE,0.999868,Connectivity Issues,Google Play
3,4,CBE,I cannot send to cbebirr app. through this app.,3,2025-06-05,NEGATIVE,0.995335,Financial Transactions,Google Play
4,5,CBE,good,4,2025-06-05,POSITIVE,0.999816,Other,Google Play


In [23]:
conn.close()